In [ ]:
%matplotlib widget
%load_ext autoreload
import numpy as np
import os 
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.figure, matplotlib.axes
import os
import sys
import pickle as pkl
import yaml
import tqdm
import collections.abc
import torch
import torch.utils.data as torchdata

thisfiledir=os.path.abspath("")
deepracingmodelsdir = os.path.abspath(os.path.join(thisfiledir, ".."))
deepracingdir = os.path.abspath(os.path.join(thisfiledir, "..", "..", "deepracing_py"))
if (not (deepracingmodelsdir in sys.path)) or (not (deepracingdir in sys.path)):
    sys.path = [deepracingmodelsdir, deepracingdir] + sys.path

homedir = os.environ["HOME"]

mtrdir=os.path.join(homedir, "deepracingws", "MTR")
print(mtrdir)
if (not (mtrdir in sys.path)):
    sys.path.insert(0, mtrdir)
print(sys.path)
class PredictionResults(collections.abc.Mapping[str,np.ndarray]):
    def __init__(self, resultsdict : dict[str,np.ndarray], data_dir : str, modelname : str) -> None:
        self.resultsdict = resultsdict
        self.data_dir = data_dir
        self.modelname = modelname
    def __iter__(self):
        return iter(self.resultsdict)
    def __len__(self):
        return int(self.resultsdict["predictions"].shape[0])
    def __getitem__(self, key):
        return self.resultsdict[key]
    def __setitem__(self, key, value):
        self.resultsdict[key] = value
    def numsamples(self):
        return self.resultsdict["predictions"].shape[0]
    def keys(self):
        return self.resultsdict.keys()
    @staticmethod
    def from_data_file(data_file : str, modelname : str, sort_idx : np.ndarray | None = None) -> 'PredictionResults':
        data_dir = os.path.dirname(data_file)
        with open(data_file, "rb") as f:
            results_file = np.load(f)
            if sort_idx is None:
                results_dict = {k: v.copy() for (k,v) in results_file.items()}
            else:
                results_dict = {k: v[sort_idx].copy() for (k,v) in results_file.items()}
        return PredictionResults(results_dict, data_dir, modelname)
    def compute_fde(self):
        self.resultsdict["fde"] = np.linalg.norm(self.resultsdict["predictions"][:,-1,[0,1]] - self.resultsdict["ground_truth"][:,-1,[0,1]], ord=2.0, axis=1)
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [ ]:
import deepracing_models.data_loading.utils.file_utils as file_utils
from deepracing_models.data_loading import SubsetFlag


keys : set = {
    "hist_vel",
    "hist",
    "fut",
    "fut_tangents",
    "fut_vel",
    "left_bd",
    "right_bd",
    "future_left_bd",
    "future_right_bd",
    "future_centerline",
    "future_raceline",
    "future_left_bd_arclength",
    "future_right_bd_arclength",
    "future_centerline_arclength",
    "future_raceline_arclength",
    "thistory",
    "tfuture"
}
datadir = "/p/DeepRacing/unpacked_datasets/local_fitting/v1/deepracing_standard"
datasets = file_utils.load_datasets_from_files(datadir, keys=keys, flag=SubsetFlag.TEST)



composite_experiment = "chosen_preservative_7505"
composite_results_dir = os.path.join("/p/DeepRacing/bamf_results", composite_experiment)
composite_results = PredictionResults.from_data_file(os.path.join(composite_results_dir, "data.npz"), "BARTé")
composite_results.compute_fde()




In [ ]:
from scipy.spatial.transform import Rotation
import deepracing_models.math_utils as mu



In [ ]:
from scipy.spatial.transform import Rotation
import deepracing_models.math_utils as mu

concat_dataset = torchdata.ConcatDataset(datasets)
reflection = np.eye(2)
reflection[0,0]*=-1.0
rot = Rotation.from_rotvec([0.0, 0.0, -np.pi/2])
Rmat = rot.as_matrix()[0:2,0:2] @ reflection
numsamples = len(composite_results)
# idxrand = np.random.randint(0, high=numsamples)
# idxrand=56815
# idxrand=33804
idxrand=5378
datasetsample = concat_dataset[idxrand]
drot = Rotation.from_rotvec([0.0, 0.0, -9.0*(np.pi/180.0)])
dR = drot.as_matrix()[0:2,0:2]
print(idxrand)
predictions = composite_results["predictions"][idxrand][:,[0,1]] @ Rmat
predictions = predictions @ dR
ground_truth = composite_results["ground_truth"][idxrand][:,[0,1]] @ Rmat
# ground_truth_vel = composite_results["ground_truth_vel"][idxrand][:,[0,1]] @ Rmat
history_vel = datasetsample["hist_vel"][:,[0,1]] @ Rmat
ground_truth_vel = datasetsample["fut_vel"][:,[0,1]] @ Rmat
history = composite_results["history"][idxrand][:,[0,1]] @ Rmat
future_left_bd = datasetsample["future_left_bd"][:,[0,1]] @ Rmat
future_right_bd = datasetsample["future_right_bd"][:,[0,1]] @ Rmat
future_centerline = datasetsample["future_centerline"][:,[0,1]] @ Rmat
future_raceline = datasetsample["future_raceline"][:,[0,1]] @ Rmat
all_points = torch.as_tensor(np.stack([
    future_left_bd,
    future_right_bd,
    future_centerline,
    future_raceline,
], axis=0)
, dtype=torch.float64)
all_arclengths = torch.as_tensor(np.stack([
    datasetsample["future_left_bd_arclength"],
    datasetsample["future_right_bd_arclength"],
    datasetsample["future_centerline_arclength"],
    datasetsample["future_raceline_arclength"],
], axis=0)
, dtype=torch.float64)
all_arclengths-=all_arclengths[:,[0,]]
Mfit, allcurves = mu.bezierLsqfit(all_points, 4, t=all_arclengths)




print(datasetsample.keys())


idx_plot = [25, -5]
future_tangents = datasetsample["fut_tangents"][:,[0,1]] @ Rmat
predictions +=  12.0*future_tangents

future_tangents = future_tangents[idx_plot]
predictions = predictions[idx_plot]

fut_tangents_full = np.zeros([future_tangents.shape[0], 3], dtype=future_tangents.dtype)
fut_tangents_full[:,:2] += future_tangents
up = np.zeros_like(fut_tangents_full)
up[:,-1] = 1.0
future_normals_full = np.cross(up, fut_tangents_full)
future_normals = future_normals_full[:,[0,1]]
# 56815
tangent_skip = 19
scatterscale=15.0
fig, _ax_ = plt.subplots(1,2, constrained_layout=True)
ax : matplotlib.axes.Axes = _ax_[0]
ax.plot(history[-40:,0], history[-40:,1], label="History", color="grey", linestyle="--")
ax.plot(ground_truth[:,0], ground_truth[:,1], label="Ground Truth", color="blue")

linealpha=0.5
linestyle="--"
lbplot = ax.plot(future_left_bd[:,0], future_left_bd[:,1], linestyle=linestyle, alpha=linealpha)
ax.scatter(allcurves[0,:,0].cpu(),  allcurves[0,:,1].cpu(), label="Left Boundary", c=lbplot[0].get_color(), s=scatterscale)

rbplot =ax.plot(future_right_bd[:,0], future_right_bd[:,1],linestyle=linestyle, alpha=linealpha)
ax.scatter(allcurves[1,:,0].cpu(),  allcurves[1,:,1].cpu(), label="Right Boundary", c=rbplot[0].get_color(), s=scatterscale)

clplot =ax.plot(future_centerline[:,0], future_centerline[:,1], linestyle=linestyle, alpha=linealpha)
ax.scatter(allcurves[2,:,0].cpu(),  allcurves[2,:,1].cpu(), label="Centerline", c=clplot[0].get_color(), s=scatterscale)

olplot =ax.plot(future_raceline[:,0], future_raceline[:,1], linestyle=linestyle, alpha=linealpha)
ax.scatter(allcurves[3,:,0].cpu(),  allcurves[3,:,1].cpu(), label="Optimal Raceline", c=olplot[0].get_color(), s=scatterscale)
pointscale = 3.0
# ax.scatter(ground_truth[idx_plot,0], ground_truth[idx_plot,1], color="blue", s=pointscale)
# ax.scatter(predictions[:,0], predictions[:,1], label="Predictions", color="red", s=pointscale)
# kwargs = {
#     "angles" : "xy",
#     # "units" : "xy",
#     "scale_units" : 'xy',
#     "scale" : 1.0,
#     "width" : 0.0125
# }
#loc="center right"
ax.legend()
ax.set_xlabel("X Position ($m$)")
ax.set_ylabel("Y Position ($m$)")
ax.set_title("Reference Paths")
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels() + ax.get_legend().get_texts()):
    item.set_fontsize(10)
frame = ax.get_legend().get_frame()
frame.set_fill(False)
# frame.set_facecolor(None)
# frame.set_edgecolor(None)


# aspect_ratio = ax.get_aspect()
# future_normals[:,1]=aspect_ratio*future_normals[:,1]
# future_tangents[:,1]=aspect_ratio*future_tangents[:,1]

# ax.set_aspect('equal')
thistory = datasetsample["thistory"]
history_speed = np.linalg.norm(history_vel, ord=2.0, axis=1)
tfuture = datasetsample["tfuture"]
ground_truth_speed = np.linalg.norm(ground_truth_vel, ord=2.0, axis=1)
pad_inches = 0.05
with plt.rc_context({"svg.fonttype": 'none', "text.usetex" : True}) as ctx:
    ax_speed : matplotlib.axes.Axes = _ax_[1]
    ax_speed.plot(thistory, history_speed, label="History", color="grey", linestyle="--")
    ax_speed.plot(tfuture, ground_truth_speed, label="Ground Truth", color="blue")
    ax_speed.legend()
    ax_speed.set_title("Speeds")
    ax_speed.set_xlabel("Time ($s$)")
    ax_speed.set_ylabel("Speed ($\\frac{m}{s}$)")
    ax_speed.legend()
    for item in ([ax_speed.title, ax_speed.xaxis.label, ax_speed.yaxis.label] +
                ax_speed.get_xticklabels() + ax_speed.get_yticklabels() + ax_speed.get_legend().get_texts()):
        item.set_fontsize(10)
# plt.axis('off')
outpath = "/bigtemp/ttw2xk/scratch/reference_lines"
pad_inches = 0.01
with plt.rc_context({"svg.fonttype": 'none', 'savefig.format' : 'png'}) as ctx:
    fig.savefig(outpath, pad_inches = pad_inches)
with plt.rc_context({"svg.fonttype": 'none', 'savefig.format' : 'svg'}) as ctx:
    fig.savefig(outpath, pad_inches = pad_inches, bbox_inches="tight", transparent=True)
with plt.rc_context({"svg.fonttype": 'none', 'savefig.format' : 'pdf'}) as ctx:
    fig.savefig(outpath, pad_inches = pad_inches, bbox_inches="tight", transparent=True)

plt.close(fig=fig)
